https://www.npci.org.in/product/ecosystem-statistics/upi

https://www.npci.org.in/api/ecosystem-statistics/get-statistics?product_name=UPI&tab_name=chargeback&year=2025&month=Jan&page_no=1&sort_by=asc&size=100&locale=en

In [3]:
import requests
import pandas as pd

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
year = 2025

bank_dict = {}

for month in months:
    url = f"https://www.npci.org.in/api/ecosystem-statistics/get-statistics?product_name=UPI&tab_name=chargeback&year={year}&month={month}&page_no=1&sort_by=asc&size=100&locale=en"
    try:
        resp = requests.get(url, timeout=15)
        data = resp.json()
        records = data.get("results", [])
        for rec in records[:15]:
            bank = rec.get("beneficiary_bank", "").strip()
            txns = int(rec.get("total_txns_during_the_month", "0").replace(",", ""))
            if bank not in bank_dict:
                bank_dict[bank] = {m:0 for m in months}
            bank_dict[bank][month] = txns
    except Exception as e:
        print(f"Month {month} error: {e}")

# Always ensure every bank gets all months as columns, even if missing data
# Build DataFrame from dict-of-dicts, fill missing with 0 and force month columns
combined_df = pd.DataFrame.from_dict(bank_dict, orient='index')
combined_df.index.name = 'Bank Name'
combined_df['Year'] = year

# If any month is missing from columns, add it and fill 0
for month in months:
    if month not in combined_df.columns:
        combined_df[month] = 0

# Reorder columns: Bank Name, Year, Jan-Dec
cols = ['Bank Name', 'Year'] + months
combined_df.reset_index(inplace=True)
combined_df = combined_df[cols]

combined_df.to_csv("upi_monthly_top15_banks_2025_wide.csv", index=False)
print("Saved upi_monthly_top15_banks_2025_wide.csv")


Saved upi_monthly_top15_banks_2025_wide.csv


In [4]:
import requests
import pandas as pd

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
year = 2025

# Collect all banks to ensure consistent columns
bank_set = set()
data_by_bank = {}

for month in months:
    url = f"https://www.npci.org.in/api/ecosystem-statistics/get-statistics?product_name=UPI&tab_name=chargeback&year={year}&month={month}&page_no=1&sort_by=asc&size=100&locale=en"
    resp = requests.get(url, timeout=15)
    api_data = resp.json()
    # Use new structure: always under 'data', 'results'
    records = api_data.get('data', {}).get('results', [])
    # Get first 15 entries
    for item in records[:15]:
        bank = item.get('beneficiary_bank', '').strip()
        txns = int(item.get('total_txns_during_the_month', '0').replace(',', ''))
        bank_set.add(bank)
        if bank not in data_by_bank:
            data_by_bank[bank] = {m: 0 for m in months}
        data_by_bank[bank][month] = txns

# Make dataframe: rows = banks, columns = months
df = pd.DataFrame.from_dict(data_by_bank, orient='index')
df.index.name = 'Bank Name'
df['Year'] = year
df = df.reset_index()

# Reorder for output: Bank Name, Year, Jan...Dec
cols = ['Bank Name', 'Year'] + months
df = df[cols]

df.to_csv("upi_monthly_top15_banks_simple.csv", index=False)
print("Saved upi_monthly_top15_banks_simple.csv")


Saved upi_monthly_top15_banks_simple.csv


In [5]:
import requests
import pandas as pd

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = list(range(2020, 2026))
rows = []

for year in years:
    # Dictionary to accumulate each bank's months for the given year
    bank_months = {}
    for month in months:
        url = f"https://www.npci.org.in/api/ecosystem-statistics/get-statistics?product_name=UPI&tab_name=chargeback&year={year}&month={month}&page_no=1&sort_by=asc&size=100&locale=en"
        try:
            resp = requests.get(url, timeout=15)
            api_data = resp.json()
            records = api_data.get('data', {}).get('results', [])
            # Get first 10 records (top 10 banks)
            for item in records[:10]:
                bank = item.get('beneficiary_bank', '').strip()
                txns = int(item.get('total_txns_during_the_month', '0').replace(',', ''))
                if bank not in bank_months:
                    bank_months[bank] = {m: 0 for m in months}
                bank_months[bank][month] = txns
        except Exception as e:
            print(f"Error for {year}-{month}: {e}")
            continue
    # For this year, output one row per bank
    for bank, month_dict in bank_months.items():
        row = {'Bank Name': bank, 'Year': year}
        row.update(month_dict)
        rows.append(row)

# All data: one row per bank per year, with Jan-Dec columns
df = pd.DataFrame(rows)
cols = ['Bank Name', 'Year'] + months
df = df[cols]
df.to_csv("bankwise_20-25_upi.csv", index=False)
print("Saved bankwise_20-25_upi.csv")


Saved bankwise_20-25_upi.csv


In [6]:
import requests
import pandas as pd

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = list(range(2020, 2026))
final_rows = []

for year in years:
    # Step 1: Collect all banks and their monthly transactions
    bank_month_txns = {}
    for month in months:
        url = f"https://www.npci.org.in/api/ecosystem-statistics/get-statistics?product_name=UPI&tab_name=chargeback&year={year}&month={month}&page_no=1&sort_by=asc&size=100&locale=en"
        try:
            resp = requests.get(url, timeout=15)
            api_data = resp.json()
            records = api_data.get('data', {}).get('results', [])
            for item in records:
                bank = item.get('beneficiary_bank', '').strip()
                txns = int(item.get('total_txns_during_the_month', '0').replace(',', ''))
                if bank not in bank_month_txns:
                    bank_month_txns[bank] = {m: 0 for m in months}
                bank_month_txns[bank][month] += txns
        except Exception as e:
            print(f"Error for {year}-{month}: {e}")
            continue
    # Step 2: Sum yearly txns, sort to get top 10
    bank_yearly_total = {bank: sum(months_dict.values()) for bank, months_dict in bank_month_txns.items()}
    top10_banks = sorted(bank_yearly_total.items(), key=lambda x: x[1], reverse=True)[:10]

    # Step 3: Output rows for top 10 banks, include monthly columns
    for bank, total in top10_banks:
        row = {'Bank Name': bank, 'Year': year}
        row.update(bank_month_txns[bank])  # Add Jan-Dec columns
        final_rows.append(row)

# Make CSV
df = pd.DataFrame(final_rows)
cols = ['Bank Name', 'Year'] + months
df = df[cols]
df.to_csv("bankwise_top10_upi.csv", index=False)
print("Saved bankwise_top10_upi.csv")

Error for 2021-Mar: invalid literal for int() with base 10: '-'
Error for 2024-Apr: 'NoneType' object has no attribute 'strip'
Saved bankwise_top10_upi.csv
